In [1]:
file_name = 'nlp_torturous_aorta.csv'

In [2]:
import os
import shutil
from collections import Counter

import pandas as pd

from config import DATASTORE


subset = pd.read_hdf(os.path.join(DATASTORE, 'store.h5'), '/subset')
subset = subset.fillna('')

## Review 100 reports randomly selected from sample10k

In [3]:
# These pattern was design by observation the reports above
pattern = r'(?i)(tortuo[^\.]*?aort(a|ic)|aort(a|ic)[^\.]*?tortuo\w+)'

In [6]:
def doctor_profile(dataset, pattern):
    """Given regex pattern, return the profile about how the doctor report this pattern
    return the doctor/count/workload/ratio 
    """
    pos = dataset[dataset['FINDINGS'].str.contains(pattern)]
    dr_who_report = pos['CONFIRMDR']
    workload = Counter(subset['CONFIRMDR']).most_common()
    profile = pd.DataFrame(workload, columns=['name', 'workload'] ).set_index('name', drop=True)
    report_count = Counter(dr_who_report).most_common()
    report_count = pd.DataFrame(report_count, columns=['name', 'count'] ).set_index('name', drop=True)
    profile = profile.assign(count=report_count).fillna(0)
    profile = profile.assign(ratio=profile['count']/profile['workload'])
    return profile.sort_values('ratio', ascending=False)

In [7]:
threshold = 0.3
profile = doctor_profile(subset, pattern)
sensitive_doctors = profile[(profile['ratio'] > threshold) & (profile['workload'] > 50)]; sensitive_doctors

/Users/jeroyang/.local/share/virtualenvs/qrabbit-nlp-2666-b4E/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """


,workload,count,ratio
name,,,
謝麗莎,90,68.0,0.755556
曾水權,939,590.0,0.628328
歐陽翊潔,171,93.0,0.543860
王裕仁,97,43.0,0.443299
陳友強,305,122.0,0.400000
賴彥君,947,326.0,0.344245
陳郁惟,52,17.0,0.326923


In [ ]:
sensitive_doctors = sensitive_doctors.index

In [ ]:
full = pd.read_hdf(os.path.join(DATASTORE, 'store.h5'), '/femh').fillna('')
relaible_reports = full[full['CONFIRMDR'].isin(sensitive_doctors)]
pos_full = full[full['FINDINGS'].str.contains(pattern)]
len(pos_full)

In [ ]:
positive_reports = pos_full
positive_reports = positive_reports.assign(label=1)

In [ ]:
pos_neg = pd.merge(relaible_reports, positive_reports, how='outer', left_on='ACCNO', right_index=True)
pos_neg['label'].fillna(0, inplace=True)
stats = Counter(pos_neg['label'])
print('正樣本:{}, 負樣本:{}'.format(stats[1.0], stats[0.0]))

In [ ]:
pos_neg[['label']].to_csv(os.path.join('data', file_name))

# QC of Image

In [ ]:
system_output = pd.read_csv(os.path.join('data', file_name), index_col=0)

In [ ]:
shutil.rmtree('data/QC/pos', ignore_errors=True)
shutil.rmtree('data/QC/neg', ignore_errors=True)
os.makedirs('data/QC/pos')
os.makedirs('data/QC/neg')

In [ ]:
has_file = pd.merge(subset, system_output, left_index=True, right_index=True)[['label']]

for accno in has_file[has_file['label']==1].sample(50).index:
    src = os.path.join(DATASTORE,'subset/{}.png'.format(accno))
    dst = 'data/QC/pos/{}.png'.format(accno)
    shutil.copy(src, dst)
    
for accno in has_file[has_file['label']==0].sample(50).index:
    src = os.path.join(DATASTORE,'subset/{}.png'.format(accno))
    dst = 'data/QC/neg/{}.png'.format(accno)
    shutil.copy(src, dst)


# Varify results